# Download dos arquivos de imagem do GeoSampa

In [13]:
import geopandas as gpd
import csv

In [14]:
gdf_scms = gpd.read_file('downloads/SIRGAS_SHP_quadriculaortofoto2017.zip!SIRGAS_SHP_quadriculaortofoto2017/SIRGAS_SHP_quadriculaortofoto2017.shp')

In [15]:
urls = []

for k, i in gdf_scms.iterrows():
    url = f"http://geosampa.prefeitura.sp.gov.br/PaginasPublicas/downloadIfr.aspx?orig=DownloadMapaArticulacao&arq=ORTOFOTOS_2017_RGB%5C{i['qo2010_cod']}.zip&arqTipo=MAPA_ARTICULACAO"
    urls.append([url])

In [16]:
with open('downloads/imagens-2017-url-download.csv', 'w') as f: 
    write = csv.writer(f)
    write.writerows(urls)

## Fazendo Download

In [12]:
# Script independente no arquivo 'download-imagens.py'

## Validando os downloads

In [75]:
import glob
import re

In [76]:
files = glob.glob('downloads/full/*.zip')

In [77]:
scms_baixados = [re.findall(r'[0-9]+-[0-9]+', f)[0] for f in files]

In [78]:
gdf_scms[~gdf_scms['qo2010_cod'].isin(scms_baixados)]['qo2010_cod']

Series([], Name: qo2010_cod, dtype: object)

In [79]:
urls = []

for i in gdf_scms[~gdf_scms['qo2010_cod'].isin(scms_baixados)]['qo2010_cod'].to_list():
    url = f"http://geosampa.prefeitura.sp.gov.br/PaginasPublicas/downloadIfr.aspx?orig=DownloadMapaArticulacao&arq=ORTOFOTOS_2017_RGB%5C{i}.zip&arqTipo=MAPA_ARTICULACAO"
    urls.append(url)

In [80]:
urls

[]

## Descomprimindo os arquivos

In [81]:
import zipfile

In [82]:
for file in files:
    with zipfile.ZipFile(file, 'r') as zip_ref:
        zip_ref.extractall('downloads/full_unzipped')

In [83]:
## Validando a descompressão

In [84]:
unzipped = glob.glob('downloads/full_unzipped/*.jp2')

In [85]:
scms_unzipped = [re.findall(r'[0-9]+-[0-9]+', f)[0] for f in unzipped]

In [87]:
len(scms_unzipped)

4250

In [88]:
gdf_scms[~gdf_scms['qo2010_cod'].isin(scms_unzipped)]['qo2010_cod']

Series([], Name: qo2010_cod, dtype: object)

## Comprimindo para enviar para o Kaggle